In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D

In [3]:
# pip install keras-tuner

In [ ]:
# def build_model(hp):  
#   model = keras.Sequential([
#     keras.layers.Conv2D(
#         filters = hp.Int('conv_1_filter', min_value = 32, max_value = 512, step = 16),
#         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
#         activation = hp.choice('activation'  , values = ['relu', 'sigmoid', 'tanh']),
#         input_shape = (256 , 256, 3)
#     ),
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_2_filter', min_value = 32, max_value = 512, step = 8 ),
#         kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
#         activation = hp.choice('activation' , values = ['relu', 'sigmoid', 'tanh']),
#     ),
#     keras.layers.Flatten(),
#     keras.layers.Dense(
#         units = hp.Int('dense_1_units', min_value = 32, max_value= 512, step = 8 ),
#         activation = hp.choice('activation' , values = ['relu', 'sigmoid', 'tanh']),
#     ),
#     keras.layers.Dense(3 , activation='softmax')
#   ])
  
#   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
  
#   return model

In [ ]:
# def build_model(hp):  
#   model = keras.Sequential([
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
#         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
#         activation='relu',
#         input_shape=(256,256, 3)
#     ),
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
#         kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
#         activation='relu'
#     ),
#     keras.layers.Flatten(),
#     keras.layers.Dense(
#         units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
#         activation='relu'
#     ),
#     keras.layers.Dense(10, activation='softmax')
#   ])
  
#   model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
  
#   return model

In [2]:
train_generator = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=10,
        horizontal_flip=True
    
)

In [3]:
train_data = train_generator.flow_from_directory(
        "Dataset/Original/train",
        target_size=(256, 256),
        class_mode="sparse",
       
)

Found 2402 images belonging to 3 classes.


In [4]:
train_data.class_indices

{'blast': 0, 'brown': 1, 'healthy': 2}

In [5]:
class_names = list(train_data.class_indices.keys())
class_names 

['blast', 'brown', 'healthy']

In [6]:
val_generator = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=10,
        horizontal_flip=True
    
)

In [7]:
val_data = val_generator.flow_from_directory(
        "Dataset/Original/val",
        target_size=(256, 256),
        class_mode="sparse",
       
)

Found 268 images belonging to 3 classes.


In [8]:
type(train_data)

keras.preprocessing.image.DirectoryIterator

In [9]:
len(train_data)

76

In [10]:
def build_model(hp):

    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', min_value = 1 , max_value = 10 )):
        
        if(counter == 0):
        
            model.add(Conv2D(filters = hp.Int('conv_filter'+str(i), min_value = 32 , max_value = 512, step = 32) , kernel_size=  hp.Choice('conv_kernel'+str(i), values = [3,5]) , activation = hp.Choice('activation'+str(i) , values = ['relu', 'tanh']), input_shape = ( 256 , 256, 3), padding= hp.Choice('Padding'+str(i), values= ['same', 'valid']) ))
            model.add(MaxPooling2D(pool_size = hp.Choice('MaxPooling'+str(i) , values = [2, 3]) ))

            model.add(Dropout(hp.Choice('Dropout'+str(i), values = [0.1, 0.2, 0.3, 0.4 , 0.5, 0.6, 0.7, 0.8, 0.9])))
            
        else:
            model.add(Conv2D(filters = hp.Int('conv_filter'+str(i), min_value = 32 , max_value = 512, step = 32) , kernel_size=  hp.Choice('conv_1_kernel'+str(i), values = [3,5]) , activation = hp.Choice('activation'+str(i) , values = ['relu', 'tanh']), input_shape = ( 256 , 256, 3),  padding= hp.Choice('Padding'+str(i), values= ['same', 'valid']) ))
            model.add(MaxPooling2D(pool_size = hp.Choice('MaxPooling'+str(i) , values = [2, 3]) ))

            model.add(Dropout(hp.Choice('Dropout'+str(i), values = [0.1, 0.2, 0.3, 0.4 , 0.5, 0.6, 0.7, 0.8, 0.9])))

        counter += 1

    model.add(Flatten())
    
    model.add(Dense(units = hp.Int('dense_1_units', min_value = 32, max_value= 512, step = 8 ),
        activation = hp.Choice('activation' , values = ['relu', 'tanh'])))
    
    model.add(Dense(3 , activation="softmax"))

    model.compile(optimizer= hp.Choice('optimizer', values = ['adam', 'adadelta', 'rmsprop', 'sgd', 'adagrad']) , loss= "sparse_categorical_crossentropy", metrics=['accuracy'])

    return model

In [11]:
import keras_tuner as kt
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [12]:
tuner = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials = 7 , directory='output', project_name="Rice Disease Classification3")

In [ ]:
tuner.search(train_data , epochs = 5, validation_data = val_data)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |?                 |num_layers
384               |?                 |conv_filter0
3                 |?                 |conv_kernel0
tanh              |?                 |activation0
same              |?                 |Padding0
2                 |?                 |MaxPooling0
0.2               |?                 |Dropout0
344               |?                 |dense_1_units
relu              |?                 |activation
adadelta          |?                 |optimizer

Epoch 1/5


In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models= 1)[0]

In [ ]:
model.fit(train_data , validation_data = val_data , epochs = 200 )

In [ ]:
model.history()